# Using "df_subset.csv" (which is subset of actual data with an unique index number for different patients) for project, above you can find basic details about dataset. 

## Normalization of database

In [1]:
### Utility Functions
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [2]:
def create_data_dic(data_filename):
    # custom made function
    # Inputs: Name of the data 
    # Output: returns data dic with key as headers and rows for those headers as values

    ### BEGIN SOLUTION
    #reading csv file and adding lines to a list. The seprate used is ",", please change if using on
    # anyother file
    header = ''
    data = []
    with open(data_filename, 'r') as file:
        for lines in file:
            if not lines.strip():
                continue
            if not header:
                header=lines.split(',')
                continue
            data.append(lines.split(","))

    ## creating a dic for headers and data
    data_dic = {}
    for count1, i in enumerate(header):
        inli = []
        for count2,j in enumerate(data):
            inli.append(j[count1].strip())
        data_dic[i.strip()] = inli

    return data_dic
    ### END SOLUTION

In [3]:
data_filename = "df_subset.csv"
normalized_database_filename = "updated_normalized.db"

In [31]:
data_dic = create_data_dic(data_filename)

Tables to be created are:

* Program Category
* Region Served
* Sex
* Living Situation
* Education Status
* Household Composition
* Employment Status
* Mental Illness
* Base patient table


In [4]:
def step1_create_program_category_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    pgm_catgrs = []
    for line in data_dic["Program Category"]:
        if line not in pgm_catgrs:
            pgm_catgrs.append(line)
    pgm_catgrs.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [ProgramCategory] (
    [ProgramID] Integer not null primary key,
    [Program] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "ProgramCategory")


    def insert_values(conn, values):
        sql = """INSERT INTO ProgramCategory(Program)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in pgm_catgrs:
            #print(count)
            insert_values(conn_norm, (count,))

    ### END SOLUTION



In [5]:
step1_create_program_category_table(data_filename, normalized_database_filename)

In [6]:
def step1_create_program_to_programid_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_program="SELECT Program FROM ProgramCategory"
    programs = execute_sql_statement(sql_select_program, conn)
    programs=list(map(lambda row: row[0].strip(), programs))
    conn.close()
    our_dict={}
    count=1
    for i in programs:
        our_dict[i]=count
        count+=1
    return our_dict

In [7]:
step1_create_program_to_programid_dictionary(normalized_database_filename)

{'EMERGENCY': 1,
 'INPATIENT': 2,
 'OUTPATIENT': 3,
 'RESIDENTIAL': 4,
 'SUPPORT': 5}

In [8]:
def step2_create_regions_served_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Region Served"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [RegionsServed] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "RegionsServed")


    def insert_values(conn, values):
        sql = """INSERT INTO RegionsServed(Region)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION



In [9]:
step2_create_regions_served_table(data_filename, normalized_database_filename)

In [10]:
def step2_create_region_to_regionid_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_region="SELECT Region FROM RegionsServed"
    regions = execute_sql_statement(sql_select_region, conn)
    regions=list(map(lambda row: row[0].strip(), regions))
    conn.close()
    our_dict={}
    count=1
    for i in regions:
        our_dict[i]=count
        count+=1
    return our_dict

In [11]:
step2_create_region_to_regionid_dictionary(normalized_database_filename)

{'CENTRAL NY REGION': 1,
 'HUDSON RIVER REGION': 2,
 'LONG ISLAND REGION': 3,
 'NEW YORK CITY REGION': 4,
 'WESTERN REGION': 5}

In [12]:
def step3_create_sex_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Sex"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [SexTable] (
    [SexID] Integer not null primary key,
    [SexValue] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "SexTable")


    def insert_values(conn, values):
        sql = """INSERT INTO SexTable(SexValue)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [13]:
step3_create_sex_table(data_filename, normalized_database_filename)

In [14]:
def step3_create_sex_to_sexid_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_sex="SELECT SexValue FROM SexTable"
    sexvalues = execute_sql_statement(sql_select_sex, conn)
    sexvalues=list(map(lambda row: row[0].strip(), sexvalues))
    conn.close()
    our_dict={}
    count=1
    for i in sexvalues:
        our_dict[i]=count
        count+=1
    return our_dict

In [15]:
step3_create_sex_to_sexid_dictionary(normalized_database_filename)

{'FEMALE': 1, 'MALE': 2, 'UNKNOWN': 3, 'X (NON-BINARY)': 4}

In [16]:
def step4_living_sit_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Living Situation"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [LivingSit] (
    [LivingID] Integer not null primary key,
    [Situation] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "LivingSit")


    def insert_values(conn, values):
        sql = """INSERT INTO LivingSit(Situation)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [17]:
step4_living_sit_table(data_filename, normalized_database_filename)

In [18]:
def step4_create_livingsitid_to_livingsit_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_living="SELECT Situation FROM LivingSit"
    livingsits = execute_sql_statement(sql_select_living, conn)
    livingsits=list(map(lambda row: row[0].strip(), livingsits))
    conn.close()
    our_dict={}
    count=1
    for i in livingsits:
        our_dict[i]=count
        count+=1
    return our_dict

In [19]:
step4_create_livingsitid_to_livingsit_dictionary(normalized_database_filename)

{'INSTITUTIONAL SETTING': 1,
 'OTHER LIVING SITUATION': 2,
 'PRIVATE RESIDENCE': 3,
 'UNKNOWN': 4}

In [20]:
def step5_education_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Education Status"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [Education] (
    [EducationID] Integer not null primary key,
    [Status] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "Education")


    def insert_values(conn, values):
        sql = """INSERT INTO Education(Status)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [21]:
step5_education_table(data_filename, normalized_database_filename)

In [22]:
def step5_create_Educationid_to_Eduation_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_education="SELECT Status FROM Education"
    edu = execute_sql_statement(sql_select_education, conn)
    edu=list(map(lambda row: row[0].strip(), edu))
    conn.close()
    our_dict={}
    count=1
    for i in edu:
        our_dict[i]=count
        count+=1
    return our_dict

In [23]:
step5_create_Educationid_to_Eduation_dictionary(normalized_database_filename)

{'COLLEGE OR GRADUATE DEGREE': 1,
 'MIDDLE SCHOOL TO HIGH SCHOOL': 2,
 'NO FORMAL EDUCATION': 3,
 'OTHER': 4,
 'PRE-K TO FIFTH GRADE': 5,
 'SOME COLLEGE': 6,
 'UNKNOWN': 7}

In [24]:
def step6_household_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Household Composition"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [Household] (
    [HouseID] Integer not null primary key,
    [Composition] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "Household")


    def insert_values(conn, values):
        sql = """INSERT INTO Household(Composition)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [25]:
step6_household_table(data_filename, normalized_database_filename)

In [26]:
def step6_create_householdid_to_household_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_household="SELECT Composition FROM Household"
    households = execute_sql_statement(sql_select_household, conn)
    households=list(map(lambda row: row[0].strip(), households))
    conn.close()
    our_dict={}
    count=1
    for i in households:
        our_dict[i]=count
        count+=1
    return our_dict

In [27]:
step6_create_householdid_to_household_dictionary(normalized_database_filename)

{'COHABITATES WITH OTHERS': 1,
 'LIVES ALONE': 2,
 'NOT APPLICABLE': 3,
 'UNKNOWN': 4}

In [28]:
def step7_employment_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Employment Status"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [Employment] (
    [EmploymentID] Integer not null primary key,
    [Status] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "Employment")


    def insert_values(conn, values):
        sql = """INSERT INTO Employment(Status)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [29]:
step7_employment_table(data_filename, normalized_database_filename)

In [30]:
def step7_create_empid_to_emp_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_employment="SELECT Status FROM Employment"
    empstatus = execute_sql_statement(sql_select_employment, conn)
    empstatus=list(map(lambda row: row[0].strip(), empstatus))
    conn.close()
    our_dict={}
    count=1
    for i in empstatus:
        our_dict[i]=count
        count+=1
    return our_dict

In [31]:
step7_create_empid_to_emp_dictionary(normalized_database_filename)

{'EMPLOYED': 1,
 'NON-PAID/VOLUNTEER': 2,
 'NOT IN LABOR FORCE:UNEMPLOYED AND NOT LOOKING FOR WORK': 3,
 'UNEMPLOYED: LOOKING FOR WORK': 4,
 'UNKNOWN EMPLOYMENT STATUS': 5}

In [32]:
def step8_mental_illness_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION

    #getting data from data file and storing in data_dic
    data_dic = create_data_dic(data_filename)
    
    #taking out values from data_dic 
    unique_list = []
    for line in data_dic["Mental Illness"]:
        if line not in unique_list:
            unique_list.append(line)
    unique_list.sort()
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [MentalIllness] (
    [IllnessID] Integer not null primary key,
    [IllnessStatus] Text not null
    );
    """
    create_table(conn_norm, create_table_sql, "MentalIllness")


    def insert_values(conn, values):
        sql = """INSERT INTO MentalIllness(IllnessStatus)
                    VALUES(?)"""
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid


    with conn_norm:
        for count in unique_list:
            insert_values(conn_norm, (count,))

    ### END SOLUTION

In [33]:
step8_mental_illness_table(data_filename, normalized_database_filename)

In [34]:
def step8_create_illnessid_to_illness_dictionary(normalized_database_filename):
    conn = sqlite3.connect(normalized_database_filename)
    sql_select_Illness="SELECT IllnessStatus FROM MentalIllness"
    illness = execute_sql_statement(sql_select_Illness, conn)
    illness=list(map(lambda row: row[0].strip(), illness))
    conn.close()
    our_dict={}
    count=1
    for i in illness:
        our_dict[i]=count
        count+=1
    return our_dict

In [35]:
step8_create_illnessid_to_illness_dictionary(normalized_database_filename)

{'NO': 1, 'UNKNOWN': 2, 'YES': 3}

In [36]:
def step_9_patient_table(data_filename, normalized_database_filename):
    
    ### BEGIN SOLUTION
    data_dic = create_data_dic(data_filename)
    programs=step1_create_program_to_programid_dictionary(normalized_database_filename)
    regions=step2_create_region_to_regionid_dictionary(normalized_database_filename)
    sexvalues=step3_create_sex_to_sexid_dictionary(normalized_database_filename)
    livingsits=step4_create_livingsitid_to_livingsit_dictionary(normalized_database_filename)
    educations=step5_create_Educationid_to_Eduation_dictionary(normalized_database_filename)
    households=step6_create_householdid_to_household_dictionary(normalized_database_filename)
    empstatuses=step7_create_empid_to_emp_dictionary(normalized_database_filename)
    illnesses=step8_create_illnessid_to_illness_dictionary(normalized_database_filename)
    patients = []
    
    for i in range(len(data_dic["index"])):
        index=data_dic["index"][i]
        program=programs[data_dic["Program Category"][i]]
        region=regions[data_dic["Region Served"][i]]
        sex=sexvalues[data_dic["Sex"][i]]
        livingsit=livingsits[data_dic["Living Situation"][i]]
        edu=educations[data_dic["Education Status"][i]]
        household=households[data_dic["Household Composition"][i]]
        emp=empstatuses[data_dic["Employment Status"][i]]
        illness=illnesses[data_dic["Mental Illness"][i]]
        insert_tuple=(index,program,region,sex,livingsit,edu,household,emp,illness)
        patients.append(insert_tuple)
        
        
    #creating DB connection
    conn_norm = create_connection(normalized_database_filename)    
    
    #creating table schema in the db
    create_table_sql = """CREATE TABLE [Patients] (
    [PatientID] Integer not null primary key,
    [ProgramID] Integer not null,
    [RegionID] Integer not null,
    [SexID] Integer not null,
    [LivingID] Integer not null,
    [EducationID] Integer not null,
    [HouseID] Integer not null,
    [EmploymentID] Integer not null,
    [IllnessID] Integer not null,
    FOREIGN KEY(ProgramID) REFERENCES ProgramCategory(ProgramID)
    FOREIGN KEY(RegionID) REFERENCES RegionsServed(RegionID)
    FOREIGN KEY(SexID) REFERENCES SexTable(SexID)
    FOREIGN KEY(LivingID) REFERENCES LivingSit(LivingID)
    FOREIGN KEY(EducationID) REFERENCES Education(EducationID)
    FOREIGN KEY(HouseID) REFERENCES Household(HouseID)
    FOREIGN KEY(EmploymentID) REFERENCES Employment(EmploymentID)
    FOREIGN KEY(IllnessID) REFERENCES MentalIllness(IllnessID)
    );
    """
    
    create_table(conn_norm, create_table_sql, "Patients")

    sql_insert_patient = '''INSERT INTO Patients (PatientID, ProgramID, RegionID, SexID, LivingID, EducationID, HouseID, EmploymentID, IllnessID)
            VALUES(?,?,?,?,?,?,?,?,?) '''
    cur = conn_norm.cursor()
    cur.executemany(sql_insert_patient,patients)
    conn_norm.commit()
    #sql_select_patients = ''' Select * from Patients'''
    #patients_print = execute_sql_statement(sql_select_patients, conn_norm)
    conn_norm.close()
    ### END SOLUTION

In [37]:
step_9_patient_table(data_filename, normalized_database_filename)